In [1]:
import os
os.environ["OPENAI_API_KEY"] = "9o0dvHP8hByOQIcJqdSTmk5JFR5wksEyfshPyj1M9LjLNQLUpZVFwCN3SwV1NxmlN5lXRbPzRVT3BlbkFJqtB0_oVZMtf3vstRBFfbAFKd6r8fo91wt5abRe1f4FeSsDlmGlIQ0Z70Hl9H3XezAZstQ32BAA"  # ⚠️ REPLACE THIS
print("✅ API Key set!")

✅ API Key set!


In [4]:
import json
import sqlite3
import hashlib
import logging
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Union
from dataclasses import dataclass, asdict
from enum import Enum
import re
import PyPDF2
from io import BytesIO
import openai
import os

In [6]:


# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class InputFormat(Enum):
    PDF = "PDF"
    JSON = "JSON"
    EMAIL = "EMAIL"

class Intent(Enum):
    INVOICE = "INVOICE"
    RFQ = "RFQ"
    COMPLAINT = "COMPLAINT"
    REGULATION = "REGULATION"
    GENERAL = "GENERAL"

@dataclass
class ProcessingResult:
    source_id: str
    format_type: InputFormat
    intent: Intent
    sender: Optional[str]
    timestamp: datetime
    extracted_data: Dict
    anomalies: List[str]
    thread_id: Optional[str] = None

class SharedMemory:
    """Lightweight shared memory using SQLite"""
    
    def __init__(self, db_path: str = "shared_memory.db"):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS processing_results (
                id TEXT PRIMARY KEY,
                format_type TEXT,
                intent TEXT,
                sender TEXT,
                timestamp TEXT,
                extracted_data TEXT,
                anomalies TEXT,
                thread_id TEXT
            )
        ''')
        conn.commit()
        conn.close()
    
    def store_result(self, result: ProcessingResult):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO processing_results 
            (id, format_type, intent, sender, timestamp, extracted_data, anomalies, thread_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            result.source_id,
            result.format_type.value,
            result.intent.value,
            result.sender,
            result.timestamp.isoformat(),
            json.dumps(result.extracted_data),
            json.dumps(result.anomalies),
            result.thread_id
        ))
        conn.commit()
        conn.close()
        logger.info(f"Stored result for {result.source_id}")
    
    def get_result(self, source_id: str) -> Optional[ProcessingResult]:
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM processing_results WHERE id = ?', (source_id,))
        row = cursor.fetchone()
        conn.close()
        
        if row:
            return ProcessingResult(
                source_id=row[0],
                format_type=InputFormat(row[1]),
                intent=Intent(row[2]),
                sender=row[3],
                timestamp=datetime.fromisoformat(row[4]),
                extracted_data=json.loads(row[5]),
                anomalies=json.loads(row[6]),
                thread_id=row[7]
            )
        return None
    
    def get_thread_history(self, thread_id: str) -> List[ProcessingResult]:
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM processing_results WHERE thread_id = ? ORDER BY timestamp', (thread_id,))
        rows = cursor.fetchall()
        conn.close()
        
        results = []
        for row in rows:
            results.append(ProcessingResult(
                source_id=row[0],
                format_type=InputFormat(row[1]),
                intent=Intent(row[2]),
                sender=row[3],
                timestamp=datetime.fromisoformat(row[4]),
                extracted_data=json.loads(row[5]),
                anomalies=json.loads(row[6]),
                thread_id=row[7]
            ))
        return results

class ClassifierAgent:
    """Central agent that classifies input format and intent"""
    
    def __init__(self, openai_api_key: str):
        openai.api_key = openai_api_key
        self.client = openai.OpenAI(api_key=openai_api_key)
    
    def classify_format(self, content: Union[str, bytes, Dict]) -> InputFormat:
        """Classify the input format"""
        if isinstance(content, dict):
            return InputFormat.JSON
        elif isinstance(content, bytes):
            # Check if it's a PDF
            try:
                PyPDF2.PdfReader(BytesIO(content))
                return InputFormat.PDF
            except:
                return InputFormat.EMAIL  # Assume email if not PDF
        elif isinstance(content, str):
            # Check if it's JSON string
            try:
                json.loads(content)
                return InputFormat.JSON
            except:
                return InputFormat.EMAIL
        else:
            return InputFormat.EMAIL
    
    def classify_intent(self, content: str, format_type: InputFormat) -> Intent:
        """Use LLM to classify intent"""
        try:
            prompt = f"""
            Analyze the following content and classify its intent into one of these categories:
            - INVOICE: Bills, invoices, payment requests
            - RFQ: Request for quotation, procurement requests
            - COMPLAINT: Customer complaints, issues, problems
            - REGULATION: Legal documents, compliance, regulations
            - GENERAL: Everything else
            
            Content ({format_type.value}):
            {content[:1000]}...
            
            Respond with only the category name.
            """
            
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,
                temperature=0.1
            )
            
            intent_str = response.choices[0].message.content.strip().upper()
            
            # Map response to enum
            for intent in Intent:
                if intent.value in intent_str:
                    return intent
            
            return Intent.GENERAL
            
        except Exception as e:
            logger.error(f"Error in intent classification: {e}")
            return Intent.GENERAL
    
    def process(self, content: Union[str, bytes, Dict], source_id: str = None) -> Tuple[InputFormat, Intent]:
        """Main classification process"""
        if source_id is None:
            source_id = hashlib.md5(str(content).encode()).hexdigest()[:8]
        
        format_type = self.classify_format(content)
        
        # Convert content to string for intent classification
        if isinstance(content, dict):
            content_str = json.dumps(content)
        elif isinstance(content, bytes):
            if format_type == InputFormat.PDF:
                # Extract text from PDF
                try:
                    pdf_reader = PyPDF2.PdfReader(BytesIO(content))
                    content_str = ""
                    for page in pdf_reader.pages:
                        content_str += page.extract_text()
                except:
                    content_str = "Unable to extract PDF text"
            else:
                content_str = content.decode('utf-8', errors='ignore')
        else:
            content_str = str(content)
        
        intent = self.classify_intent(content_str, format_type)
        
        logger.info(f"Classified {source_id}: Format={format_type.value}, Intent={intent.value}")
        return format_type, intent

class JSONAgent:
    """Agent for processing JSON payloads"""
    
    def __init__(self, target_schema: Dict = None):
        self.target_schema = target_schema or {
            "required_fields": ["id", "type", "data"],
            "optional_fields": ["metadata", "timestamp"]
        }
    
    def extract_and_reformat(self, json_data: Dict) -> Tuple[Dict, List[str]]:
        """Extract and reformat JSON to target schema"""
        extracted_data = {}
        anomalies = []
        
        # Check required fields
        for field in self.target_schema["required_fields"]:
            if field in json_data:
                extracted_data[field] = json_data[field]
            else:
                anomalies.append(f"Missing required field: {field}")
        
        # Extract optional fields
        for field in self.target_schema["optional_fields"]:
            if field in json_data:
                extracted_data[field] = json_data[field]
        
        # Check for unexpected fields
        expected_fields = set(self.target_schema["required_fields"] + self.target_schema["optional_fields"])
        for field in json_data:
            if field not in expected_fields:
                anomalies.append(f"Unexpected field: {field}")
        
        # Validate data types and values
        if "id" in extracted_data and not isinstance(extracted_data["id"], (str, int)):
            anomalies.append("ID field should be string or integer")
        
        return extracted_data, anomalies
    
    def process(self, json_data: Dict, source_id: str) -> ProcessingResult:
        """Process JSON data"""
        extracted_data, anomalies = self.extract_and_reformat(json_data)
        
        return ProcessingResult(
            source_id=source_id,
            format_type=InputFormat.JSON,
            intent=Intent.GENERAL,  # Will be updated by classifier
            sender=json_data.get("sender", "unknown"),
            timestamp=datetime.now(),
            extracted_data=extracted_data,
            anomalies=anomalies
        )

class EmailAgent:
    """Agent for processing email content"""
    
    def __init__(self, openai_api_key: str):
        openai.api_key = openai_api_key
        self.client = openai.OpenAI(api_key=openai_api_key)
    
    def extract_email_info(self, email_content: str) -> Dict:
        """Extract structured information from email"""
        # Basic regex patterns for email parsing
        sender_pattern = r'From:\s*([^\n\r]+)|From\s*[<\(]([^>\)]+)[>\)]'
        subject_pattern = r'Subject:\s*([^\n\r]+)'
        
        sender_match = re.search(sender_pattern, email_content, re.IGNORECASE)
        subject_match = re.search(subject_pattern, email_content, re.IGNORECASE)
        
        sender = sender_match.group(1) or sender_match.group(2) if sender_match else "unknown"
        subject = subject_match.group(1) if subject_match else "No subject"
        
        # Use LLM to extract additional info
        try:
            prompt = f"""
            Analyze this email and extract:
            1. Urgency level (LOW/MEDIUM/HIGH)
            2. Key topics (comma-separated)
            3. Action items (if any)
            4. Sentiment (POSITIVE/NEUTRAL/NEGATIVE)
            
            Email content:
            {email_content[:1500]}
            
            Respond in JSON format:
            {{"urgency": "...", "topics": "...", "action_items": "...", "sentiment": "..."}}
            """
            
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0.3
            )
            
            llm_analysis = json.loads(response.choices[0].message.content)
            
        except Exception as e:
            logger.error(f"Error in LLM analysis: {e}")
            llm_analysis = {
                "urgency": "MEDIUM",
                "topics": "general",
                "action_items": "none",
                "sentiment": "NEUTRAL"
            }
        
        return {
            "sender": sender.strip(),
            "subject": subject.strip(),
            "urgency": llm_analysis.get("urgency", "MEDIUM"),
            "topics": llm_analysis.get("topics", "general"),
            "action_items": llm_analysis.get("action_items", "none"),
            "sentiment": llm_analysis.get("sentiment", "NEUTRAL"),
            "word_count": len(email_content.split()),
            "crm_format": {
                "contact": sender.strip(),
                "subject": subject.strip(),
                "priority": llm_analysis.get("urgency", "MEDIUM"),
                "category": llm_analysis.get("topics", "general").split(',')[0].strip()
            }
        }
    
    def process(self, email_content: str, source_id: str) -> ProcessingResult:
        """Process email content"""
        extracted_data = self.extract_email_info(email_content)
        
        # Generate thread ID based on sender and subject
        thread_key = f"{extracted_data['sender']}_{extracted_data['subject']}"
        thread_id = hashlib.md5(thread_key.encode()).hexdigest()[:12]
        
        return ProcessingResult(
            source_id=source_id,
            format_type=InputFormat.EMAIL,
            intent=Intent.GENERAL,  # Will be updated by classifier
            sender=extracted_data["sender"],
            timestamp=datetime.now(),
            extracted_data=extracted_data,
            anomalies=[],
            thread_id=thread_id
        )

class MultiAgentSystem:
    """Main orchestrator for the multi-agent system"""
    
    def __init__(self, openai_api_key: str):
        self.classifier = ClassifierAgent(openai_api_key)
        self.json_agent = JSONAgent()
        self.email_agent = EmailAgent(openai_api_key)
        self.memory = SharedMemory()
    
    def process_input(self, content: Union[str, bytes, Dict], source_id: str = None) -> ProcessingResult:
        """Main processing pipeline"""
        if source_id is None:
            source_id = hashlib.md5(str(content).encode()).hexdigest()[:8]
        
        # Step 1: Classify format and intent
        format_type, intent = self.classifier.process(content, source_id)
        
        # Step 2: Route to appropriate agent
        if format_type == InputFormat.JSON:
            if isinstance(content, str):
                content = json.loads(content)
            result = self.json_agent.process(content, source_id)
        elif format_type == InputFormat.EMAIL:
            if isinstance(content, bytes):
                content = content.decode('utf-8', errors='ignore')
            result = self.email_agent.process(str(content), source_id)
        elif format_type == InputFormat.PDF:
            # For PDF, extract text and treat as email-like content
            try:
                pdf_reader = PyPDF2.PdfReader(BytesIO(content))
                text_content = ""
                for page in pdf_reader.pages:
                    text_content += page.extract_text()
                result = self.email_agent.process(text_content, source_id)
            except Exception as e:
                logger.error(f"Error processing PDF: {e}")
                result = ProcessingResult(
                    source_id=source_id,
                    format_type=format_type,
                    intent=intent,
                    sender="unknown",
                    timestamp=datetime.now(),
                    extracted_data={"error": str(e)},
                    anomalies=["PDF processing failed"]
                )
        
        # Update intent from classifier
        result.intent = intent
        
        # Step 3: Store in shared memory
        self.memory.store_result(result)
        
        return result
    
    def get_processing_history(self, source_id: str = None, thread_id: str = None) -> List[ProcessingResult]:
        """Get processing history"""
        if source_id:
            result = self.memory.get_result(source_id)
            return [result] if result else []
        elif thread_id:
            return self.memory.get_thread_history(thread_id)
        else:
            return []

# Example usage and testing
if __name__ == "__main__":
    # Initialize system (you'll need to set your OpenAI API key)
    api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")
    system = MultiAgentSystem(api_key)
    
    # Test with different input types
    print("Multi-Agent AI System Demo")
    print("=" * 50)
    
    # Test JSON input
    json_input = {
        "id": "INV-001",
        "type": "invoice",
        "data": {"amount": 1500, "vendor": "ABC Corp"},
        "sender": "finance@abccorp.com"
    }
    
    print("\n1. Processing JSON input:")
    result1 = system.process_input(json_input, "json_test_1")
    print(f"Result: {result1.format_type.value} - {result1.intent.value}")
    print(f"Extracted: {result1.extracted_data}")
    
    # Test Email input
    email_input = """
    From: customer@example.com
    Subject: Urgent: Issue with recent order
    
    Dear Support Team,
    
    I am writing to complain about the defective product I received yesterday.
    The item was damaged and doesn't match the description. Please provide
    immediate assistance as this is affecting my business operations.
    
    Best regards,
    John Doe
    """
    
    print("\n2. Processing Email input:")
    result2 = system.process_input(email_input, "email_test_1")
    print(f"Result: {result2.format_type.value} - {result2.intent.value}")
    print(f"Sender: {result2.sender}")
    print(f"Thread ID: {result2.thread_id}")
    
    print("\n" + "=" * 50)
    print("Demo completed successfully!")

Multi-Agent AI System Demo

1. Processing JSON input:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.442834 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.813529 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified json_test_1: Format=JSON, Intent=GENERAL
INFO:__main__:Stored result for json_test_1


Result: JSON - GENERAL
Extracted: {'id': 'INV-001', 'type': 'invoice', 'data': {'amount': 1500, 'vendor': 'ABC Corp'}}

2. Processing Email input:


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.384055 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.956691 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified email_test_1: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too

Result: EMAIL - GENERAL
Sender: customer@example.com
Thread ID: 6aa0c9c5175a

Demo completed successfully!


In [8]:
api_key = os.getenv("OPENAI_API_KEY")
system = MultiAgentSystem(api_key)
print("🚀 System ready!")

🚀 System ready!


In [10]:
# Quick test
test_data = {"id": "TEST-001", "type": "invoice", "sender": "test@example.com"}
result = system.process_input(test_data)
print(f"SUCCESS: {result.format_type.value} - {result.intent.value}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.466003 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.757602 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified 32d6ae7a: Format=JSON, Intent=GENERAL
INFO:__main__:Stored result for 32d6ae7a


SUCCESS: JSON - GENERAL


In [12]:
# Test if your main system is working first
print("🧪 Testing main system...")
test_result = system.process_input({"test": "data"})
print(f"✅ System working: {test_result.format_type.value}")

🧪 Testing main system...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.423481 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.958770 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified a537caca: Format=JSON, Intent=GENERAL
INFO:__main__:Stored result for a537caca


✅ System working: JSON


In [16]:
# Simple Testing Interface for Jupyter (No widgets needed!)

def test_multi_agent_system():
    """Simple function-based interface for testing"""
    
    print("🤖 MULTI-AGENT SYSTEM TESTER")
    print("=" * 50)
    
    # Test 1: JSON Processing
    print("\n1️⃣ TESTING JSON PROCESSING...")
    json_test = {
        "id": "INV-2024-001",
        "type": "invoice",
        "vendor": "TechCorp Solutions", 
        "amount": 2500.00,
        "sender": "billing@techcorp.com"
    }
    
    try:
        result1 = system.process_input(json_test, "json_demo")
        print(f"✅ SUCCESS: {result1.format_type.value} - {result1.intent.value}")
        print(f"👤 Sender: {result1.sender}")
        print(f"📊 Extracted: {result1.extracted_data}")
        print(f"⚠️ Anomalies: {result1.anomalies}")
    except Exception as e:
        print(f"❌ JSON Error: {e}")
    
    # Test 2: Email Processing  
    print("\n2️⃣ TESTING EMAIL PROCESSING...")
    email_test = """From: customer@company.com
Subject: Urgent complaint about defective product

Dear Support Team,

I am extremely disappointed with my recent order #12345. 
The product arrived completely damaged and unusable.

This is the third issue I've had with your products and I need 
immediate resolution or I will escalate to management.

Please respond within 24 hours.

Angry regards,
John Smith"""

    try:
        result2 = system.process_input(email_test, "email_demo")
        print(f"✅ SUCCESS: {result2.format_type.value} - {result2.intent.value}")
        print(f"👤 Sender: {result2.sender}")
        print(f"🧵 Thread ID: {result2.thread_id}")
        print(f"📊 Urgency: {result2.extracted_data.get('urgency', 'N/A')}")
        print(f"😡 Sentiment: {result2.extracted_data.get('sentiment', 'N/A')}")
    except Exception as e:
        print(f"❌ Email Error: {e}")
    
    # Test 3: RFQ Email
    print("\n3️⃣ TESTING RFQ PROCESSING...")
    rfq_test = """From: procurement@bigcorp.com
Subject: RFQ - Office Equipment Q2 2024

Dear Vendor,

We need quotations for:
- 50 Office chairs 
- 25 Standing desks
- Printer supplies

Please send quotes by March 15th.

Best regards,
Sarah Johnson
Procurement Manager"""

    try:
        result3 = system.process_input(rfq_test, "rfq_demo")
        print(f"✅ SUCCESS: {result3.format_type.value} - {result3.intent.value}")
        print(f"👤 Sender: {result3.sender}")
        print(f"📋 Topics: {result3.extracted_data.get('topics', 'N/A')}")
    except Exception as e:
        print(f"❌ RFQ Error: {e}")
    
    # Test 4: Check Memory Storage
    print("\n4️⃣ CHECKING SHARED MEMORY...")
    try:
        stored = system.memory.get_result("json_demo")
        if stored:
            print(f"✅ Memory working: Found {stored.source_id}")
            print(f"📅 Stored at: {stored.timestamp}")
        else:
            print("❌ Memory issue: No stored result found")
    except Exception as e:
        print(f"❌ Memory Error: {e}")
    
    print("\n" + "=" * 50)
    print("🎉 TESTING COMPLETE!")

# Interactive testing functions
def test_custom_json():
    """Test with your own JSON"""
    print("🔧 CUSTOM JSON TESTER")
    print("Copy-paste your JSON below, then run process_my_json(your_data)")
    
def process_my_json(json_data):
    """Process your custom JSON"""
    try:
        result = system.process_input(json_data)
        print(f"✅ Result: {result.format_type.value} - {result.intent.value}")
        print(f"📊 Data: {result.extracted_data}")
        print(f"⚠️ Issues: {result.anomalies}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

def test_custom_email():
    """Test with your own email"""
    print("📧 CUSTOM EMAIL TESTER") 
    print("Copy-paste your email below, then run process_my_email('your_email_text')")

def process_my_email(email_text):
    """Process your custom email"""
    try:
        result = system.process_input(email_text)
        print(f"✅ Result: {result.format_type.value} - {result.intent.value}")
        print(f"👤 Sender: {result.sender}")
        print(f"🧵 Thread: {result.thread_id}")
        print(f"📊 Analysis: {result.extracted_data}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Quick demo data for easy testing
DEMO_DATA = {
    "invoice_json": {
        "id": "INV-001",
        "type": "invoice", 
        "amount": 1500,
        "vendor": "ABC Corp",
        "sender": "finance@abc.com"
    },
    
    "complaint_email": """From: frustrated@customer.com
Subject: URGENT - Product defect complaint

This is the worst service ever! My order #999 is completely wrong 
and I demand immediate refund and compensation!

Very angry,
Customer""",
    
    "rfq_email": """From: buyer@company.com
Subject: Request for Quote - IT Equipment  

Please provide quotes for 20 laptops and 5 printers.
Deadline: Next Friday.

Thanks,
IT Manager"""
}

print("🚀 SIMPLE INTERFACE LOADED!")
print("\n📋 Available Functions:")
print("• test_multi_agent_system() - Run all tests")
print("• process_my_json(data) - Test your JSON")  
print("• process_my_email('text') - Test your email")
print("• DEMO_DATA - Pre-made test data")
print("\n✨ Ready to test! Run: test_multi_agent_system()")

🚀 SIMPLE INTERFACE LOADED!

📋 Available Functions:
• test_multi_agent_system() - Run all tests
• process_my_json(data) - Test your JSON
• process_my_email('text') - Test your email
• DEMO_DATA - Pre-made test data

✨ Ready to test! Run: test_multi_agent_system()


In [18]:
test_multi_agent_system()

🤖 MULTI-AGENT SYSTEM TESTER

1️⃣ TESTING JSON PROCESSING...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.444637 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.837805 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified json_demo: Format=JSON, Intent=GENERAL
INFO:__main__:Stored result for json_demo


✅ SUCCESS: JSON - GENERAL
👤 Sender: billing@techcorp.com
📊 Extracted: {'id': 'INV-2024-001', 'type': 'invoice'}
⚠️ Anomalies: ['Missing required field: data', 'Unexpected field: vendor', 'Unexpected field: amount', 'Unexpected field: sender']

2️⃣ TESTING EMAIL PROCESSING...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.444792 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.901633 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified email_demo: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too M

✅ SUCCESS: EMAIL - GENERAL
👤 Sender: customer@company.com
🧵 Thread ID: 87f73be1b37b
📊 Urgency: MEDIUM
😡 Sentiment: NEUTRAL

3️⃣ TESTING RFQ PROCESSING...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.426984 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.901180 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:__main__:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:__main__:Classified rfq_demo: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Man

✅ SUCCESS: EMAIL - GENERAL
👤 Sender: procurement@bigcorp.com
📋 Topics: general

4️⃣ CHECKING SHARED MEMORY...
✅ Memory working: Found json_demo
📅 Stored at: 2025-05-29 21:30:14.887528

🎉 TESTING COMPLETE!


In [3]:
import json
import os

# Create sample_inputs directory
os.makedirs("sample_inputs", exist_ok=True)

# Sample JSON files
sample_json_invoice = {
    "id": "INV-2024-001",
    "type": "invoice",
    "vendor": "TechCorp Solutions",
    "amount": 2500.00,
    "currency": "USD",
    "due_date": "2024-02-15",
    "issue_date": "2024-01-15",
    "items": [
        {
            "description": "Software License Annual Subscription",
            "quantity": 5,
            "unit_price": 400.00,
            "total": 2000.00
        },
        {
            "description": "Setup and Configuration",
            "quantity": 1,
            "unit_price": 500.00,
            "total": 500.00
        }
    ],
    "sender": "billing@techcorp.com",
    "recipient": "accounts@yourcompany.com",
    "metadata": {
        "payment_terms": "Net 30",
        "purchase_order": "PO-2024-005"
    }
}

sample_json_rfq = {
    "id": "RFQ-2024-Q1-001",
    "type": "request_for_quotation",
    "category": "office_supplies",
    "deadline": "2024-02-01",
    "requirements": [
        {
            "item": "A4 Paper",
            "specification": "80gsm, white",
            "quantity": 100,
            "unit": "reams"
        },
        {
            "item": "Printer Cartridges",
            "specification": "HP LaserJet Pro series, black",
            "quantity": 20,
            "unit": "pieces"
        },
        {
            "item": "Office Chairs",
            "specification": "Ergonomic, adjustable height, black",
            "quantity": 25,
            "unit": "pieces"
        }
    ],
    "sender": "procurement@company.com",
    "delivery_address": "123 Business Park, Suite 100, City, State 12345",
    "contact_person": "Sarah Johnson",
    "phone": "+1-555-0123"
}

sample_json_malformed = {
    "id": 12345,  # Should be string
    "data": "Some data",
    "unknown_field": "This shouldn't be here",
    # Missing required 'type' field
    "sender": "test@example.com"
}

# Sample Email contents
sample_email_complaint = """From: angry.customer@email.com
To: support@company.com
Subject: URGENT - Defective Product Complaint - Order #12345
Date: Mon, 15 Jan 2024 14:30:00 GMT

Dear Customer Service Team,

I am writing to express my extreme disappointment with my recent purchase from your company. Order #12345, placed on January 5th, arrived yesterday in completely unacceptable condition.

Issues encountered:
1. The main product unit was visibly damaged with scratches and dents
2. Several components were missing from the package
3. The instruction manual was water-damaged and illegible
4. The product doesn't match the specifications listed on your website

This is the THIRD time I've had quality issues with your products, and my patience is completely exhausted. As a business owner, these delays and defects are causing serious disruptions to my operations.

I DEMAND:
- Immediate replacement of the entire order with quality-checked items
- Full refund of shipping costs for both deliveries
- Compensation for business losses caused by these delays
- Escalation to management level for review of your quality control processes

If this matter is not resolved within 24 hours, I will be forced to:
- File complaints with consumer protection agencies
- Share my negative experience on social media and review platforms
- Consider legal action for breach of contract

I expect immediate action and a detailed response plan.

Furiously disappointed,
John Smith
CEO, Smith Enterprises
Phone: 555-987-6543
Email: john@smithenterprises.com"""

sample_email_rfq = """From: procurement@company.com
To: vendor@supplier.com
Subject: RFQ - Office Equipment Q1 2024 - Response Required by Feb 1st
Date: Tue, 16 Jan 2024 09:15:00 GMT

Dear Vendor Partner,

We are soliciting quotations for our Q1 2024 office equipment procurement. Please review the requirements below and provide your competitive pricing.

ITEMS REQUESTED:

1. Desktop Computers
   - Specification: Intel i5 processor, 16GB RAM, 512GB SSD, Windows 11 Pro
   - Quantity: 15 units
   - Required delivery: March 1, 2024

2. Monitors
   - Specification: 24-inch LED, 1920x1080, HDMI/DisplayPort connectivity
   - Quantity: 15 units

3. Wireless Keyboards and Mice Combos
   - Specification: Ergonomic design, 2.4GHz wireless, battery included
   - Quantity: 15 sets

4. Network Switches
   - Specification: 24-port Gigabit Ethernet, managed, rack-mountable
   - Quantity: 2 units

QUOTATION REQUIREMENTS:
- Unit prices and total costs
- Warranty terms and duration
- Delivery timeline and logistics
- Payment terms and conditions
- Technical support availability

EVALUATION CRITERIA:
- Competitive pricing (40%)
- Quality and reliability (30%)
- Delivery timeline (20%)
- After-sales support (10%)

Please submit your complete quotation by February 1st, 2024, 5:00 PM EST.
Late submissions will not be considered.

For technical clarifications, contact our IT team at it@company.com
For commercial queries, reach out to me directly.

Best regards,
Sarah Johnson
Senior Procurement Manager
Company Name Inc.
Phone: +1-555-0123
Email: procurement@company.com"""

sample_email_general = """From: employee@company.com
To: team@company.com
Subject: Team Meeting Agenda - Weekly Sync
Date: Wed, 17 Jan 2024 08:45:00 GMT

Hi Team,

Hope everyone is doing well! Here's the agenda for our weekly team sync scheduled for Friday at 2:00 PM.

AGENDA ITEMS:
1. Project status updates (15 mins)
2. Q1 goals review (10 mins)
3. Upcoming deadlines discussion (10 mins)
4. Resource allocation planning (15 mins)
5. Open floor for questions/concerns (10 mins)

Please come prepared with:
- Your current project status
- Any blockers or challenges you're facing
- Resource requirements for next week

The meeting will be held in Conference Room B, with dial-in option available.
Meeting link: https://company.zoom.us/j/123456789

Looking forward to our productive discussion!

Best,
Alex Martinez
Team Lead
alex@company.com"""

# Write JSON samples
with open("sample_inputs/invoice.json", "w") as f:
    json.dump(sample_json_invoice, f, indent=2)

with open("sample_inputs/rfq.json", "w") as f:
    json.dump(sample_json_rfq, f, indent=2)

with open("sample_inputs/malformed.json", "w") as f:
    json.dump(sample_json_malformed, f, indent=2)

# Write email samples
with open("sample_inputs/complaint_email.txt", "w") as f:
    f.write(sample_email_complaint)

with open("sample_inputs/rfq_email.txt", "w") as f:
    f.write(sample_email_rfq)

with open("sample_inputs/general_email.txt", "w") as f:
    f.write(sample_email_general)

print("Sample input files created successfully!")
print("\nGenerated files:")
print("- sample_inputs/invoice.json")
print("- sample_inputs/rfq.json") 
print("- sample_inputs/malformed.json")
print("- sample_inputs/complaint_email.txt")
print("- sample_inputs/rfq_email.txt")
print("- sample_inputs/general_email.txt")

Sample input files created successfully!

Generated files:
- sample_inputs/invoice.json
- sample_inputs/rfq.json
- sample_inputs/malformed.json
- sample_inputs/complaint_email.txt
- sample_inputs/rfq_email.txt
- sample_inputs/general_email.txt
